#Intro

This Jupyter Notebook has been created for the <a href="https://www.unibo.it/it/didattica/insegnamenti/insegnamento/2021/443749" target="_blank">90154 - Electronic Publishing and Digital Storytelling</a> course, taught by **Prof. Marilena Daquino**, in the framework of the 2nd year of the <a href="https://corsi.unibo.it/2cycle/DigitalHumanitiesKnowledge" target="_blank">DHDK Master Degree</a>, a.a. 2021-22.<br>
Here listed the main steps for the realization of the project **Partizione Antica**:     
       
    1. Data Preparation:
          - creation of two complexive xml files for F and OA records coming from the Federico Zeri Foundation catalogues
          - extraction of relevant information for the project from nested xml stucture and structuring them in plain tabular format
          - extraction from previpus tabular data of unstructured annotations
    2. Data Elaboration: seeking for furter analysis elements via:
          - deeper work on photographer for enhancing their information (workplace, timespan of activity, etc.)
          - deeper work on places for enhancing their gelocation
          - work on unstructured annotations trough NLP and NER
    3. Data Visualization
    - 
    -
    -

# 1. Data preparation

This research started from a **record data extraction** of the Supino Partizione Antica fund provided from the Federico Zeri Foundation: the original data counted 3.260 records for photographs and 2.634 records for depicted works of art. <br>
The original data have been used for **illustrative and didactical purposes only**: all the credits and reuse authorizations must be asked to <a href="mailto:fondazionezeri.fototeca@unibo.it">Federico Zeri Foundation</a>.

**1.1 Creation of the F and OA complexive xml files**

To allow a better management and manipulation, as well as to anonymize personal data, complexive files (via <a href="/content/sample_data/0_Creation_UniqeXML.xquery" target="_blank">0_Creation_UniqeXML.xquery</a> collection command) have been created and published. 
They collect:

*   all the single photograph xml files' records in the F_entries.xml file (data/0_source_data)
*   all the single works of art xml files' records in the OA_entries.xml file (data/0_source_data)

**1.2 Creation of the flat tabular dataset extracting relevant information for the project from the nested xml elements and attributes**

Due to the hypernested and not consistently presence of elements at different levels, <pandas.read_xml> method was not effectively parsing what was needed.
The <xml.etree.ElementTree> library has then been preferred because it allows to call for single elements at different nesting levels. Nevertheless, this approach presents some drawbacks as the need of a previous and deep knowledge of the database structure that does not allow to uncover unexpected correlations possible through the exploration of a comprehensive dataset.

**1.3 Preliminary installation**(Uncomment the first line to install the library)
- libraries
- imports:
  - xml.etree.ElementTree, pandas, csv for managing the dataset
  - ...


In [3]:
#preliminary imports
!pip install python-csv
!pip install elementpath
import csv
import xml.etree.ElementTree as ET

#function to have back the element texts
def extract_data(path):
    if SCHEDA.find(path) != None:
        name = SCHEDA.find(path).text
    else:
        name = None
    return name

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 24.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.5/131.5 kB 21.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [1 lines of output]
      error in demjson setup command: use_2to3 is invalid.
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.1/189.1 kB 8.6 MB/s eta 0:00:00


In [9]:
#parse the complexive Fxml and OAxml files
F_tree = ET.parse("data/0_source_data/F_entries.xml")
F_root = F_tree.getroot()
F_root.attrib["test"]

OA_tree = ET.parse("data/0_source_data/OA_entries.xml")
OA_root = OA_tree.getroot()
OA_root.attrib["test2"]

#set the colums' headers for the choosen elements
header = ["sercdf_F_ser", "sercdoa_OA_ser", "INVN_F", "UBFC_Fshelfmark", #ids
          "PVCS_OAcountry", "PVCC_OAtown", "LDCN_OArep", "PRVC_OAprev_town", "AUFI_Fatelier_address", #places
          "AUFN_Faut", "SGLT_Ftitle", "SGTT_OAtitle", "AUTN_OAaut", #authors/titles
          "OGTT_OAtype", "AUTB_Fsubj_main", "OGTDOA_OAsubj_sub", #subjects
          "ROFI_Fneg", "BIBA_OAbib",#external relations
          "OSS_Fnotes", "OSS_OAnotes", #unstructured infos
         "FTAN_filename", "NCTN_F_entry", "NRSCHEDA_OA_entry", #2ary ids
        "DTSI_OAdate", "LRD_Fshotdates", "DTSI_Fprintdates", "DTSF_Fprintdates", "AUFA_Faut_dates"] #time

#setting an empty list
data = []

#iterate on F_entries - and on correspondig OA_entries -
#for extracting elements texts, store them in a list and add it to the data
#two fields from original data are futherly modify for our purposes

for SCHEDA in F_root.findall("SCHEDA"):
    oa_ser = SCHEDA.get("sercdoa")
    f_ser = SCHEDA.get("sercdf")
    inv = extract_data("./PARAGRAFO/INVN")
    container = extract_data("./PARAGRAFO/UBFT")
    shelf = extract_data("./PARAGRAFO/UBFC")
    title_f = extract_data("./PARAGRAFO/RIPETIZIONE/SGLT")
    aut_f = extract_data("./PARAGRAFO/RIPETIZIONE/AUFN") #the original data do not distinguish AUFN and AUFB for collective agents
    aut_f_dates = extract_data("./PARAGRAFO/RIPETIZIONE/AUFA")#timespan of photographer's actvity
    aut_f_addr = extract_data("./PARAGRAFO/RIPETIZIONE/AUFI")#place of photographer's actvity as reported in the photograph >AF of variants
    aut_oa = extract_data("./PARAGRAFO/RIPETIZIONE/AUTN")
    subj_main = extract_data("./PARAGRAFO/RIPETIZIONE/AUTB")
    subj_sub = extract_data("./PARAGRAFO/RIPETIZIONE/OGTDOA")
    notes_f = extract_data("./PARAGRAFO/OSS")
    neg_num = extract_data("./PARAGRAFO/ROFI")
    f_entry = extract_data("./PARAGRAFO/NCTN")
    filename = extract_data("./PARAGRAFO/FTAN")
    shotdates = extract_data("./PARAGRAFO/LRD")
    if shotdates != None:
        #reduce uncertainty: if /ante in field, put 1855 as conventional beginning date
        #for collodium negatives (accordign to other Zeri cataloguing)
        if "/ante" in shotdates:
            shotdates.replace("/ante", "/ ante")
        if "/ ante" in shotdates:
        #if re.match("/ante|\/ ante", shotdates):
            shotdates = "1855-"+shotdates[:-6]
    printdates_start = extract_data("./PARAGRAFO/DTSI")
    printdates_end = extract_data("./PARAGRAFO/DTSF")

    for SCHEDA in OA_root.findall("SCHEDA"):
        if SCHEDA.get("sercdoa") == oa_ser:
            title_oa = extract_data("./PARAGRAFO/SGTT")
            date_from_oa = extract_data("./PARAGRAFO/DTSI")
            date_to_oa = extract_data("./PARAGRAFO/DTSF")
            country_oa = extract_data("./PARAGRAFO/PVCS") #Original data report just 2 LRCS: name of the country where the shot was taken.
            town_oa = extract_data("./PARAGRAFO/PVCC") #Original data report just 2 LRCC: name of the country where the shot was taken.
            rep_oa = extract_data("./PARAGRAFO/LDCN")
            prev_town_oa = extract_data("./PARAGRAFO/RIPETIZIONE/PRVC")
            if prev_town_oa != None:
                #save the previous locations only if in 1800-1899 timespan (PRDU) otherwise put "NR" (not relevant)
                if extract_data("./PARAGRAFO/RIPETIZIONE/PRDU") != None:
                    if "1799" < extract_data("./PARAGRAFO/RIPETIZIONE/PRDU") < "1900": #PRDU last date the OA was in that location
                        prev_town_oa = prev_town_oa + " | " + str(extract_data("./PARAGRAFO/RIPETIZIONE/PRDU"))
                    else:
                        prev_town_oa = "NR"
            type_oa = extract_data("./PARAGRAFO/OGTT")
            notes_oa = extract_data("./PARAGRAFO/OSS")
            oa_entry = extract_data("./PARAGRAFO/NRSCHEDA")
            beg_date_oa = extract_data("./PARAGRAFO/DTSI")
            if extract_data("./PARAGRAFO/RIPETIZIONE/BIBA") != None:
                #save the bib ref only if in 1800-1899 timespan (BIBD) otherwise put "NR" (not relevant)
                if extract_data("./PARAGRAFO/RIPETIZIONE/BIBD") != None:
                    if "1799" < extract_data("./PARAGRAFO/RIPETIZIONE/BIBD") < "1900": #PRDU last date the OA was in that location
                        bib_oa = extract_data("./PARAGRAFO/RIPETIZIONE/BIBA")
                        bib_oa = bib_oa + " | " + str(extract_data("./PARAGRAFO/RIPETIZIONE/BIBD"))
                    else:
                        bib_oa = "NR"
            else:
                bib_oa = None

    row = [oa_ser, f_ser, inv, shelf,
           country_oa, town_oa, rep_oa, prev_town_oa, aut_f_addr,
           aut_f, title_f, title_oa, aut_oa,
           type_oa, subj_main, subj_sub,
           neg_num, bib_oa,
           notes_f, notes_oa,
           filename, f_entry, oa_entry,
           beg_date_oa, shotdates, printdates_start, printdates_end, aut_f_dates]
    data.append(row)

#Write the data and their header in a new csv dataset
with open("data/F_OA_selected_data.csv", "w", encoding="utf-8", newline="") as tabular_data:
    # create the csv writer
    writer = csv.writer(tabular_data)
    writer.writerow(header)
    writer.writerows(data)

In [2]:
!pip install pandas
import pandas as pd

In [5]:
data_df = pd.read_csv('data/F_OA_selected_data.csv')

#reduce the dataset to just the columns needed, the not-empty and not-duplicates rows
OAnotes_df = data_df[["OSS_OAnotes"]].dropna()
OAnotes_df = OAnotes_df.drop_duplicates()

#split multilines rows and once again remove duplicates rows
OAnotes_df["OSS_OAnotes"] = OAnotes_df["OSS_OAnotes"].str.split("&#10;|"". Foto ", expand = False)
OAnotes_df = OAnotes_df.explode("OSS_OAnotes")
OAnotes_df = OAnotes_df.drop_duplicates()

#save just rows with transcriptions notes (including "Foto sup \d{1,4}" string)
OAnotes_df = OAnotes_df[OAnotes_df["OSS_OAnotes"].str.contains("sup \d{1,4}")== True].reset_index(drop=True)
OAnotes_df = OAnotes_df[OAnotes_df["OSS_OAnotes"].str.startswith("La foto ")== False].reset_index(drop=True)

#separe note texts from other infos and remove the column containing the whole infos, save and check the result
OAnotes_df[["Inv", "Note"]] = OAnotes_df["OSS_OAnotes"].str.split(': "', n=1, expand=True)
OAnotes_df= OAnotes_df.drop(columns=["OSS_OAnotes"]).reset_index(drop=True)
print("Photographs which annotations have been transcribed in OA entries: ", OAnotes_df.shape[0], "(/over 3.222 photographs") #1839
print(OAnotes_df.head(10))
OAnotes_df.to_csv("data/1_working_data/1_OAnotes01.csv", encoding="utf-8")

Photographs which annotations have been transcribed in OA entries:  1839 (/over 3.222 photographs
                                                 Inv  \
0                  Foto sup 748, verso: nota anonima   
1      Foto sup 763, verso: nota anonima manoscritta   
2      Foto sup 982, verso: nota anonima manoscritta   
3  Foto sup 893, verso: nota anonima manoscritta:...   
4      Foto sup 988, verso: nota anonima manoscritta   
5      Foto sup 991, verso: nota anonima manoscritta   
6      Foto sup 998, verso: nota manoscritta anonima   
7     Foto sup 1000, verso: nota anonima manoscritta   
8     Foto sup 1002, verso: nota anonima manoscritta   
9     Foto sup 1003, verso: nota anonima manoscritta   

                                                Note  
0  Near Avezzano and not far from Tagliacozzo. He...  
1  Aquila. S. Maria di Collemaggio. Founded by Pi...  
2  The pulpit of San Giovanni del Toro, of the mi...  
3                                               None  
4  Here al

In [7]:
#manual checking and adjusting for 1)"manoscritta:">"manoscritta"," 2)"".",>""." 3)\n",>"
#inv: 943, 75, 559, 1635, 1648, 1702, (1768 non riporta), 1787, 2397, 2789, 2869, 2849,
# 2984, 2222,2270,2880, saved in data\OAnotes02.csv

#open the manually modified dataframe, search for unsuseful informations in 'Inv' and eliminate them
OAnotes2_df = pd.read_csv('data/1_working_data/1_OAnotes02.csv', encoding="utf-8").dropna(subset=['Inv']).reset_index(drop=True)
pattern = 'Foto |, (.+)'
OAnotes2_df["Inv"] = OAnotes2_df["Inv"].replace(to_replace=pattern, value='', regex=True).reset_index(drop=True)

#check and save the third version of OAnotes_df
print(OAnotes2_df.head(15))
OAnotes2_df.to_csv("data/1_working_data/1_OAnotes03.csv", encoding="utf-8")

#check how many of them are incomplete
annotations_incompleted = OAnotes2_df[OAnotes2_df['Note'].str.contains("[...]")== True].reset_index(drop=True)
print("Photographs which transcribed annotations are likely to be incomplete: ", annotations_incompleted.shape[0], "(/over ",OAnotes_df.shape[0]," transcribed)")

#create the corpus to be passed with spacy
corpus = ""
for OAnote in OAnotes2_df["Note"]:
    corpus = corpus+"---"+str(OAnote)+"---\n"
with open("data/OAnotes_corpus.txt", "w", encoding="utf-8") as f:
        f.write(corpus)

    Unnamed: 0       Inv                                               Note
0            0   sup 748  Near Avezzano and not far from Tagliacozzo. He...
1            1   sup 763  Aquila. S. Maria di Collemaggio. Founded by Pi...
2            2   sup 982  The pulpit of San Giovanni del Toro, of the mi...
3            3   sup 893  Clara pudicicie dux Paulabianca potentis / A g...
4            4   sup 988  Here also his buried Sibylla of Burgundy. "Rex...
5            5   sup 991  Queen Margherita widow of Carlo III, who died ...
6            6   sup 998  Piissimi Patris Nicolai Piscicelli optimi pres...
7            7  sup 1000  Rude sarcophagus in the porch of the church. T...
8            8  sup 1002  Amalfi. This campanile is said to date from 11...
9            9  sup 1003  Cloister of the Canonica founded in 1213 by Ca...
10          10  sup 1012  In cloister of Amalfi Duomo. Sarc. of an archb...
11          11  sup 1005  Amalfi. Cloister of San Francesco founded by t...
12          

# 2. Data elaboration




1. Work on photographers

In [3]:
!pip install SPARQLWrapper
!pip install geopy
from csv import DictReader
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import ssl
from geopy.geocoders import Nominatim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.3/500.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.3/98.3 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 6.2 MB/s eta 0:00:00


In [4]:
ssl._create_default_https_context = ssl._create_unverified_context
geolocator = Nominatim(timeout=10, user_agent="myGeolocator")

In [5]:
# functions
# define a function to open file in reading mode
def process_csv(data_file_path):
    import csv
    source = open(data_file_path, mode="r", encoding="UTF8")
    source_reader = csv.DictReader(source)
    source_data = list(source_reader)
    return source_data

#define a function for transforming lists of elements in strings
def write_string(source, output_txt_name):
    string = ""
    for source_data in source:
        string = string+source_data+"|"
    string = string[:-1]
    with open(output_txt_name, "w", encoding="utf8") as f:
        f.write(string)
    return string

#define a function to query endpoints
def query_endpoint(endpoint_url, SPRQL_query):
    get_endpoint = endpoint_url
    sparql_w = SPARQLWrapper(get_endpoint)
    sparql_w.setQuery(SPRQL_query)
    sparql_w.setReturnFormat(JSON)
    spqrl_w_res = sparql_w.query().convert()
    return spqrl_w_res

#define a function to manipulate results and have back 1. a set of wd_URI corresponding to our wd_names,
# 2. update of ph_matrix, 3. not matched wd_names

def manipulate(spqrl_w_res, dataset_to_enhance, resNF_txt_name, resF_txt_name):
    res_all = set()
    res_dic = {}
    res_NF_tem = set()
    res_F = set()
    for res in spqrl_w_res["results"]["bindings"]:
        res_all.add((res["fLabel"]["value"], res["f"]["value"]))
        for datum in dataset_to_enhance:
            if datum["ph_wd_URI"]:
                continue
            else:
                if datum["ph_wd_name"] not in res_dic:
                    if res["fLabel"]["value"] == datum["ph_wd_name"]:
                        res_F.add(res["f"]["value"])
                        new_pairs = {"ph_wd_URI": res["f"]["value"]}
                        res_dic.update({datum["ph_wd_name"]: new_pairs})
                        datum.update([("ph_wd_URI", res["f"]["value"])])
                    else:
                        res_NF_tem.add(datum["ph_wd_name"])
    res_NF_def = res_NF_tem - set(list(res_dic.keys()))
    with open(resNF_txt_name, "w", encoding="utf8") as f:
        f.write(str(res_NF_def))
    with open(resF_txt_name, "w", encoding="utf8") as f:
        f.write(str(res_F))
    print("labels matched: ", len(res_F))
    print("labels not found2: ", len(res_NF_def))
    return res_F, res_NF_def #, res_all

In [6]:
import pandas as pd
# open source data with pandas
data_df = pd.read_csv("data/F_OA_selected_data.csv")

#initialize a photograph's frequency dataframe
ph_freq = pd.DataFrame(data_df["AUFN_Faut"].value_counts().reset_index().values, columns=["AUFN_Faut", "count"])

#extend dataframe colums to host next datas
ph_freq["ph_wd_name"], ph_freq["ph_wd_URI"], ph_freq["gender"], ph_freq["workplace"], ph_freq["lat"], ph_freq["lon"],\
ph_freq["born"], ph_freq["died"], ph_freq["lat"] = ["", "", "", "", "", "", "", "", ""]

In [8]:
#create the firts string for the SPARQL query by 
#normalizing (personal) names in form "surname, name" to "name surname" as in wikidata
#and create a list of the modified names tobe added to the dataframe

first_ph_names_string =""
ph_wd_name_list = []
for ph in ph_freq.index:
    ph_name = str(ph_freq["AUFN_Faut"][ph])
    # reverse only (personal) names in form "surname, name" > "name surname"
    if ", " in ph_name:
        ph_split = ph_name.split(", ")
        ph_wd_name = ph_split[1] + " " + ph_split[0]
    else:
        ph_wd_name = ph_name
    ph_wd_name_list.append(ph_wd_name)
    first_ph_names_string = first_ph_names_string + ph_wd_name + "|"
first_ph_names_string = first_ph_names_string[:-1]

#show a sample of the string
print(first_ph_names_string[0:200])

Anonimo|Fratelli Alinari|Romualdo Moscioni|Brogi|Giorgio Sommer|Jean Laurent|Incorpora|Giraudon|Paolo Lombardi|Naya|Carlo Baldassarre Simelli|Pietro Poppi|Séraphin-Médéric  Mieusement|Robert Rive|John


In [12]:
#add ph_wd_name_list to the dataframe and show a sample of the current dataframe
ph_freq["ph_wd_name"] = ph_wd_name_list
print(ph_freq.head(10))
#save the dataframe in a csv file and open it as a dictionary to iterate
ph_freq.to_csv("data/1_working_data/2_PH_freq_01.csv", encoding="utf-8")

            AUFN_Faut count         ph_wd_name ph_wd_URI gender workplace lat  \
0             Anonimo  1336            Anonimo                                  
1   Alinari, Fratelli   556   Fratelli Alinari                                  
2  Moscioni, Romualdo   159  Romualdo Moscioni                                  
3               Brogi   158              Brogi                                  
4     Sommer, Giorgio   147     Giorgio Sommer                                  
5       Laurent, Jean    73       Jean Laurent                                  
6           Incorpora    57          Incorpora                                  
7            Giraudon    55           Giraudon                                  
8     Lombardi, Paolo    53     Paolo Lombardi                                  
9                Naya    48               Naya                                  

  lon born died  
0                
1                
2                
3                
4                


In [14]:
ph_matrix = process_csv("data/1_working_data/2_PH_freq_01.csv")

#prepare the first query string to collect wikidata URI
first_ph_SPARQL_query = """
SELECT DISTINCT ?f ?fLabel
WHERE
{    { ?f wdt:P106 wd:Q33231 } UNION { ?f wdt:P31 wd:Q672070}. #P106_has_for_occupation wd:Q33231_photographer 
                                                                #P31_is instance wd:Q672070_studios
    ?f rdfs:label ?fLabel.
     FILTER regex(?fLabel, \" """+first_ph_names_string+""" \")
     FILTER(LANG(?fLabel) = "en").
}"""

#perform the first SPARQL query and result manipulation
first_ph_wd_res = query_endpoint("https://query.wikidata.org/bigdata/namespace/wdq/sparql", first_ph_SPARQL_query)
first_ph_manipulate = manipulate(first_ph_wd_res, ph_matrix, "ph_NF.txt", "ph_F.txt")
first_F_set = first_ph_manipulate[0]
first_NF = first_ph_manipulate[1]

labels matched:  45
labels not found2:  67


In [ ]:
#after revising first results, refine the unmatched labels
new_list = []
for ph_wd_NF in first_NF:
    if "  " in ph_wd_NF:
        ph_wd_new = ph_wd_NF.replace("  ", " ")
    elif "Fratelli" in ph_wd_NF:
        ph_wd_new = ph_wd_NF.replace("Fratelli", "")
    elif "&" in ph_wd_NF:
        ph_wd_new = ph_wd_NF.replace("&", "and")
    elif "Brogi" == ph_wd_NF:
        ph_wd_new = "Giacomo Brogi"
    elif "Incorpora" == ph_wd_NF:
        ph_wd_new = "Giuseppe Incorpora"
    elif "Giraudon" == ph_wd_NF:
        ph_wd_new = "Adolphe Giraudon"
    else:
        continue
    new_list.append(ph_wd_new)
    for ph_data in ph_matrix:
        if ph_data["ph_wd_name"] == ph_wd_NF:
            ph_data.update([("ph_wd_name", ph_wd_new)])

#from the new modified names, by using the function obtain a second string to query 
second_ph_string = write_string(new_list, "ph_string2.txt")

In [ ]:
#prepare the second query string to collect wikidata URI
second_ph_SPARQL_query = """
SELECT DISTINCT ?f ?fLabel
WHERE
{    { ?f wdt:P106 wd:Q33231 } UNION { ?f wdt:P31 wd:Q672070}. #P106_has_for_occupation wd:Q33231_photographer 
                                                                #P31_is instance wd:Q672070_studios
    ?f rdfs:label ?fLabel.
     FILTER regex(?fLabel, \" """+second_ph_string+""" \")
     FILTER(LANG(?fLabel) = "en").
}
"""
#perform the second SPARQL query and result manipulation
second_ph_wd_res = query_endpoint("https://query.wikidata.org/bigdata/namespace/wdq/sparql", second_ph_SPARQL_query)
second_manipulate = manipulate(second_ph_wd_res, ph_matrix, "ph_NF2", "ph_F2.txt")
second_F_set = second_manipulate[0]

In [ ]:
#obtain the list of found wikidata URI
complex_F_set = second_F_set.union(first_F_set)
#print(complex_F_set)

#prepare the thirtd string to be passed in SPARQL query
third_ph_string_URI =""
for F_URI in complex_F_set:
    third_ph_string_URI = third_ph_string_URI+"<"+F_URI+">"
#with open("ph_URI.txt", "w", encoding="utf8") as f:
#    f.write(stringURI)
#print(stringURI)

#third query
third_ph_SPARQL_query = """
SELECT DISTINCT ?ph ?genderLabel ?countryLabel ?birthyear ?deathyear
    WHERE
    { VALUES ?ph {"""+stringURI+"""} 
        ?ph rdfs:label ?phLabel;
        wdt:P937 ?country; #P937_worklocation
        #wdt:P27 ?citiz;        
        wdt:P569 ?birth;
        wdt:P570 ?death.
        OPTIONAL {FILTER(LANG(?fLabel) = "en").
                    ?ph wdt:P21 ?gender;
                    #wdt:P937 ?worklocation; #P937_worklocation
        }
        BIND(year(?birth) AS ?birthyear)
        BIND(year(?death) AS ?deathyear)

        #BIND(COALESCE(?worklocation, ?citiz, "NaN") AS ?country).
        #BIND(IF(BOUND(?worklocation),?worklocation,?citiz) AS ?country).
    SERVICE wikibase:label {bd:serviceParam wikibase:language "en".}     
    }"""
#OPTIONAL { ?ph wdt:P569 ?birthdate;        wdt:P570 ?deathdate.} ci servono...

#perform the third query
third_ph_wd_res = query_endpoint("https://query.wikidata.org/bigdata/namespace/wdq/sparql", third_ph_SPARQL_query)

#manipulate results
wd_total_dic = {}
for result in third_ph_wd_res["results"]["bindings"]:
    item_key = result["ph"]["value"]
    item_value = {"workplace": result["countryLabel"]["value"],
                  "born": result["birthyear"]["value"],
                  "died": result["deathyear"]["value"]}
    if item_key not in wd_total_dic:
        wd_total_dic.update({item_key: item_value})
        for ph_data in ph_matrix:
            if ph_data["ph_wd_URI"] == item_key:
                item2=item_value.items()
                ph_data.update(item2)

wd_total_list = list(wd_total_dic.values())
print(wd_total_list)

keys = wd_total_list[0].keys()
with open("ph_wd_total.csv", "w", encoding="UTF8", newline="") as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(wd_total_list)

2. Work on places

In [ ]:
#define function to store lat-lon from a list of places
def get_coordinates(list, df):
    for place in list:
        if place not in df["place"].unique().tolist():
            if geolocator.geocode(place) != None:
                lon = geolocator.geocode(place).longitude
                lat = geolocator.geocode(place).latitude
            else:
                lon = "NaN"
                lat = "NaN"
            new_place = [place, lat, lon]
            df.loc[len(df)] = new_place
        df.to_csv("data\places_coordinates.csv", encoding="UTF-8")

In [ ]:
#open the saved file
data_df = pd.read_csv("data\F_OA_selected_data.csv")
ph_freq_df = pd.read_csv("data\ph_freq2.csv")

#reduce columns and change column name, check first rows
places_F = ph_freq_df[['workplace', "lat", "lon"]].dropna()
places = places_F.rename(columns={"workplace": "place"})
places.head()

#extract towns and country unique names from original dataframe
towns_OA = data_df['PVCC_OAtown'].unique().tolist()
countries_OA = data_df['PVCS_OAcountry'].unique().tolist()

#obtain coordinates from the two list and store them in a df
get_coordinates(countries_OA, places)
get_coordinates(towns_OA, places)

3. Work on Annotations

In [ ]:
#!pip install spacy
import spacy
from spacy.matcher import Matcher
from spacy.attrs import POS

In [ ]:
#open the file with annotations texts
with open("data/OAnotes_corpus.txt", mode="r") as f:
    contents = f.read()
 
#load the nlp model and look for {"LEMMA": "I"}, {POS: 'VERB'} pattern (first person verbs)
nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)
matcher.add("PA_creator", [[{"LEMMA": "I"}, {POS: 'VERB'}]])
doc = nlp(contents)
matches = matcher(doc)

matched = []
for match_id,start,end in matches:
    I_verb = str(doc[start:end])
    matched.append(I_verb)
print(matched)
#I doubt, I was, I think(3), I discovered, I have(4), I saw(4), I AM(2?), I told, I respected, I farn, me look,
# I believe, I put

In [ ]:
#create a dataframe from list of matched and check occurrencies
matched_df = pd.DataFrame()
matched_df["Match"] = matched
matched_freq = pd.DataFrame(matched_df["Match"].value_counts().reset_index().values, columns=["Match", "count"])
print(matched_freq.head(25))

# 3. Data visualization


---
**Analyse**
pandas library in order to examine our data.
     
       
    1. Data Preparation:
          - creation of two complexive xml files for F and OA records coming from the Federico Zeri Foundation catalogues
          - extraction from nested xml stucture of relevant information for the project and structuring them in plain tabular format
    2. Data Elaboration: seeking for furter analysis elements via:
          - deeper work on photographer for enhance their information
          - deeper work on places
          - work on unstructured annotations: NER
     2. Data Visualization


0. Data overview

In [ ]:
import pandas as pd
import pandas_profiling as pp
import matplotlib.pyplot as plt
import seaborn as sns
import pprint
pp = pprint.PrettyPrinter(indent=1)

In [ ]:
# parse the csv into a dataframe
data_df = pd.read_csv('data\F_OA_selected_data.csv')

# reduce the dataset to just the columns needed
data_df = data_df[['INVN_F', 'PVCS_OAcountry', 'PVCC_OAtown', 'LDCN_OArep', 'PRVC_OAprev_town',
          'AUFN_Faut', 'LRD_Fdates', 'OGTT_OAtype', 'AUTB_Fsubj_main', 'OGTDOA_OAsubj_sub', 'AUTN_OAaut']]
print(data_df.head(15))

In [ ]:
#import pandas_profiling as pp
report = pp.ProfileReport(data_df, title="Partizione Antica Fund - overview")
report.to_file("ProfileReport_sup.html")

In [ ]:
report

1.1 Works of art - typology proportions pie

In [ ]:
import pandas as pd
import plotly.express as px

# filter cities and number of photos from data
data = pd.read_csv("data/F_OA_selected_data.csv", encoding="UTF-8")
OAt_df = pd.DataFrame(data["OGTT_OAtype"].value_counts().reset_index().values, columns=["OGTT_OAtype", "count"])

fig = px.pie(OAt_df, values='count', names="OGTT_OAtype",
            title='OA typologies',
            color_discrete_sequence=px.colors.sequential.RdBu,
            labels = OAt_df['OGTT_OAtype'], hover_name = 'OGTT_OAtype',
            hover_data = {'OGTT_OAtype':False}
            )
fig.show()
fig.write_html("data/2_data_viz/1.1.html")

1.2 Works of art - countries proportions pie



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import plotly
import plotly.graph_objects as go
import csv

# filter cities and number of photos from data
data = pd.read_csv("UNIQUE_new2.csv", encoding="utf8")
sup_df = data[['PVCS_OAcountry', 'PVCC_OAtown', 'LDCN_OArep']]

#create a pie chart of supposed OA country
df_data = pd.read_csv("OAcountry_freq.csv", encoding="utf8")
df_data.loc[df_data['count'] < 10, 'PVCS_OAcountry'] = 'Other countries' # Represent only large countries
#df = px.df_data()
fig = px.pie(df_data, values='count', names="PVCS_OAcountry",
            title='Depicted OA for country',
            color_discrete_sequence=px.colors.sequential.RdBu,
            labels = df_data['PVCS_OAcountry'], hover_name = 'PVCS_OAcountry',
            hover_data = {'PVCS_OAcountry':False, 'lat':False, 'lon': False}
            )
fig.show()
fig.write_html("data/2_data_viz/1.2.html")

1.3 Works of art - countries proportions map

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from geopy.geocoders import Nominatim
geolocator = Nominatim(timeout=10, user_agent = "myGeolocator")

import plotly
import plotly.graph_objects as go
import csv

# filter cities and number of photos from data
df_data = pd.read_csv("OAcountry_freq.csv", encoding="utf8")
ph_geol = px.scatter_mapbox(df_data, lon=df_data['lon'],
                            lat=df_data['lat'], size=df_data["count"], zoom=2, color=df_data['PVCS_OAcountry'],
                            color_continuous_scale=px.colors.cyclical.Twilight,
                            #color_discrete_sequence=px.colors.sequential.RdBu,
                            title="Depicted OA",
                            size_max=80,
                            labels=df_data['PVCS_OAcountry'], hover_name='PVCS_OAcountry',
                            hover_data={'PVCS_OAcountry':False, 'lat':False, 'lon':False})

# mapbox style
ph_geol.update_layout(mapbox_style='carto-positron')
ph_geol.show()
ph_geol.write_html("data/2_data_viz/1.3.html")

2.1 Photograps - photographers proportions pie /distribution barchart

In [ ]:
import pandas as pd
import plotly.express as px

df_data2 = pd.read_csv("ph_newfreq.csv", encoding="utf8")
df_data2 = df_data2[df_data2["count"]>=20]
#df_data2.loc[df_data2['count'] < 10, 'AUFN_Faut'] = 'Other photographs' # Represent only large countries
#df = px.df_data()
fig2 = px.pie(df_data2, values='count', names="AUFN_Faut",
            title='Photographs (>=20) for photographer',
            color_discrete_sequence=px.colors.qualitative.Dark24, #color_discrete_sequence/color_continuous_scale =px.colors.sequential.RdBu,
            labels = df_data2['AUFN_Faut'], hover_name='AUFN_Faut',
            hover_data = {'AUFN_Faut':False, 'workplace':True}
            #sistema le caselle non piene di ph_freq etc penso fillna()
            )
fig2.show()
fig2.write_html("data/2_data_viz/2.1.html")

2.2 Photographs - map distribution based on ateliers locations

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
#to show up everything directly in jupyter notebook

import plotly
import plotly.graph_objects as go
import csv

#open the whole dataset and select just the AUFN_FAUT column
data = pd.read_csv("data\F_OA_selected_data.csv", encoding="UTF-8")
data_d = data[["AUFN_Faut"]]
print(len(data_d))

#open the ph_freq dataset and select just the AUFN_FAUT and the workplace column; create a second df on workplace lat and lon
data_ph = pd.read_csv("data\ph_freq.csv", encoding="UTF-8")
data_ph_df = data_ph[["AUFN_Faut", "workplace"]]
data_lat_lon = data_ph[["workplace", "lat", "lon"]].drop_duplicates(subset=['workplace'])

#merge the two dataset on the AUFN_Faut column
data_comp = data_d.merge(data_ph_df, how='left', on="AUFN_Faut").reset_index(drop=True)
print(len(data_comp))
print(data_comp.head(25))

#count the values on workplace column
data_count = pd.DataFrame(data_comp['workplace'].value_counts().reset_index().values, columns=['workplace', 'count'])
print(len(data_count))
print(data_count.head(25))

#add lat and lon data to the previous df
data_df = data_count.merge(data_lat_lon, how='left', on="workplace").reset_index(drop=True)
data_df['count'] = pd.to_numeric(data_df['count'])
print(len(data_df))
print(data_df.head(25))

ph_geol = px.scatter_mapbox(data_df, lon=data_df['lon'], lat=data_df['lat'],
                            size=data_df['count'], zoom=3, color=data_df['workplace'],
                            size_max=80,
                            color_discrete_sequence=px.colors.sequential.RdBu,
                            title="Photohgrapher ateliers",
                            labels=data_df['workplace'], hover_name="workplace")

# mapbox style
ph_geol.update_layout(mapbox_style='carto-positron')
ph_geol.show()
ph_geol.write_html("data/2_data_viz/2.2.html")

2.3 Photographs - map distribution of anonimous photographs based on place of shooting (limited to immobles)

In [ ]:
#!pip install regex
import pandas as pd
import plotly.express as px
import regex as re

data = pd.read_csv("UNIQUE_new2.csv", encoding="UTF-8")

#reduce to only columns rows needed and check
sup_df = data[['OGTT_OAtype', 'PVCS_OAcountry', 'PVCC_OAtown', 'AUFN_Faut']]
sup_df = sup_df[sup_df.AUFN_Faut == "Anonimo"]

df_data3 = pd.DataFrame(sup_df["PVCS_OAcountry"].value_counts().reset_index().values, columns=["PVCS_OAcountry", "count"])

fig3 = px.pie(df_data3, values='count', names="PVCS_OAcountry",
              title='Anonimous Photographs (1331/3111) for OAcountry',
              color_discrete_sequence=px.colors.sequential.Brwnyl,
              labels = df_data3['PVCS_OAcountry'], hover_name='PVCS_OAcountry',
              hover_data = {'PVCS_OAcountry':False}
              )

fig3.update_layout(
    font = dict(
        family="Courier New, monospace",
        size=18,
        )
    )
fig3.show()

sup_df= sup_df[sup_df['OGTT_OAtype'].str.contains("architettura|architettura\ scultura|complesso archeologico|sito archeologico")== True].reset_index(drop=True)
sup_df.to_csv("daje.csv")
print(sup_df.tail(200))
df_data4 = pd.DataFrame(sup_df["PVCS_OAcountry"].value_counts().reset_index().values, columns=["PVCS_OAcountry", "count"])
fig4 = px.pie(df_data4, values='count', names="PVCS_OAcountry",
              title='Anonimous Photographs (1331/3111) for OAcountry of immobles',
              color_discrete_sequence=px.colors.sequential.Brwnyl, #px.colors.sequential.RdBu https://plotly.com/python/discrete-color/
              labels = df_data3['PVCS_OAcountry'], hover_name='PVCS_OAcountry',
              hover_data = {'PVCS_OAcountry':False}  #sistema le caselle non piene di ph_freq etc penso fillna()
              )

fig4.update_layout(
    font = dict(
        family="Courier New, monospace",
        size=18,
        )
    )
fig4.show()
fig4.write_html("data/2_data_viz/2.3.html")

3.1 Annotations - complete/incomplete/missing transcriptions proportions pie

In [ ]:
import plotly.express as px
import matplotlib.pyplot as plt
import pandas as pd

#open the needed dataframes
data_df = pd.read_csv('data\F_OA_selected_data.csv')
all_inv = data_df[["INVN_F"]].rename(columns={"INVN_F": "Inv"}).reset_index(drop=True)
OAnotes_df = pd.read_csv('data/1_working_data/OAnotes03.csv')

#define a function to check status of transcriptions
def check(row):
    if "..." in str(row["Note"]):
        status = "incomplete"
    else:
        status = "complete"
    return status

#apply the function to df and add a status column, check it 
OAnotes_df["status"] = OAnotes_df.apply(check, axis=1)
print(OAnotes_df.head(25))

#merge the df with all the inventories to check for missing transcriptions and count according to the status 
merged = all_inv.merge(OAnotes_df, how='left', on="Inv").reset_index(drop=True)
new=pd.DataFrame(merged["status"].value_counts(dropna=False).reset_index().values, columns=["status", "count"])
#change the empty rows with "missing" and check
new['status'] = new['status'].fillna('missing')
print(new.tail(50))

fig = px.pie(new, values='count', names="status",
              title='Annotations on photographs',
              color_discrete_sequence=px.colors.sequential.Brwnyl, #px.colors.sequential.RdBu https://plotly.com/python/discrete-color/
              labels = new['status'], hover_name='status',
              hover_data = {'status':True}  #sistema le caselle non piene di ph_freq etc penso fillna())
             )

fig.update_layout(
    font = dict(
        family="Courier New, monospace",
        size=18,
        )
    )
fig.show()
fig.write_html("data/2_data_viz/3.1.html")

3.2 Annotations

3.3 Annotations and metadata: compared dates distribution

4.1 Annotations - time-place pairs related to ...

In [ ]:
import plotly.express as px
import pandas as pd

#open and merge data from annotations reporting time-place pairs and place coordinates, check for it
movement_df = pd.read_csv("data/1_working_data/OAnotes05bis.csv", encoding="utf8")
places_df = pd.read_csv("data/1_working_data/places_coordinates2.csv", encoding="utf8")
movement_df_coor = movement_df.merge(places_df, how='left', on="place")
movement_df_coor.drop('Unnamed: 0', axis=1, inplace=True)
print(movement_df_coor.head(10))

#break the texts at about 30 characters to let them better visualized
movement_df_coor['Note_br'] = movement_df_coor.apply(lambda row: ('<br>'.join(str(row.Note)[i:i+30] for i in range(0, len(str(row.Note)), 30))), axis = 1)

#set scatter with d
fig = px.scatter_geo(movement_df_coor, color="date",
                  lat=movement_df_coor["lat"].values.tolist(),
                  lon=movement_df_coor["lon"].values.tolist(),
                  title="Movements", size="date",
                  projection="natural earth", scope="europe",
            labels = movement_df_coor['place'], hover_name='place',
            hover_data = {'place':False, 'Inv':True, 'Note_br':True}
            )
fig.show()
fig.write_html("data/2_data_viz/4.1.html")